#1 관련 유튜브 동영상들의 지표를 수집

In [1]:
from apiclient.discovery import build
import pandas as pd
import numpy as np
import datetime
import pytz
import warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings(action='ignore')

In [2]:
#날짜별 조건을 만족하는 영상 ID를 추출하는 함수
def stockVideoIDExtract(pd,index,stockName,month,date,resultCnt):
    
    youtube = build("youtube", "v3", developerKey="INPUT_YOUR_KEY")
    
    AfterMonth = month
    AfterDate = date-1
    BeforeMonth = month
    BeforeDate = date
    
    date_mapping = {1:31,2:31,3:28,4:31,5:30,6:31,7:30,8:31,9:31,10:30,11:31,12:30}
    
    if month < 8 : 
        AfterYear = 2021
        BeforeYear = 2021
    else:
        AfterYear = 2020
        BeforeYear = 2020
    
    if date == 1: 
        if month == 1: 
            AfterMonth = 12
            AfterYear = 2020
            BeforeYear = 2021
        else:
            AfterMonth = month - 1
        AfterDate = date_mapping[month]
        
    #UTC 기준으로 우리나라 시각은 9시간 빠르기 때문에 이를 반영한 시간 설정
    search_response = youtube.search().list(
        q= stockName+"주식",
        part="id",#snippet
        order = "viewCount",
        publishedAfter = datetime.datetime(AfterYear, AfterMonth, AfterDate, 15, 0, 0, 0).isoformat("T") + "Z",
        publishedBefore = datetime.datetime(BeforeYear, BeforeMonth, BeforeDate, 14, 59, 59, 0).isoformat("T") + "Z",
        maxResults=resultCnt
    ).execute()
    for search_result in search_response.get("items", []):
        #응답 결과가 비디오 정보인 경우 <- 이 경우만 사용함
        if search_result["id"]["kind"] == "youtube#video":
          pd['Stock'][index] = stockName
          pd['Date'][index] = datetime.date(BeforeYear, month, date).strftime('%Y-%m-%d') 
          pd['Video ID'][index] = search_result["id"]["videoId"]
          #pd['Title'][index] = search_result["snippet"]["title"]
        #응답 결과가 채널 정보인 경우 
        elif search_result["id"]["kind"] == "youtube#channel":
          titleList.append("%s (%s)" % (search_result["snippet"]["title"],search_result["id"]["channelId"]))
        #응답 결과가 플레이리스트 정보인 경우
        elif search_result["id"]["kind"] == "youtube#playlist":
          playlists.append("%s (%s)" % (search_result["snippet"]["title"],search_result["id"]["playlistId"]))
        index = index+1

In [3]:
#영상ID를 이용하여 조회수,좋아요/싫어요수,댓글수를 추출해서 데이터프레임에 저장하는 함수
def stockVideoStatExtract(pd1):
    
    pd2 = pd1.copy()
    youtube = build("youtube", "v3", developerKey="INPUT_YOUR_KEY")

    for i in range(len(pd1)): 
        request = youtube.videos().list(
            part="statistics",
            id = pd1['Video ID'][i]
        )
        
        response = request.execute()
        
        if response['items']==[]:
            pd2['View'][i] = np.NaN
            pd2['Like'][i] = np.NaN 
            pd2['Dislike'][i] = np.NaN 
            pd2['Comment'][i] = np.NaN 
            
        else:
            #반환되는 동영상 지표들 중 각 지표들의 정보를 반환하는지 확인하고 값 저장
            
            #조회수 저장
            if 'viewCount' in response['items'][0]['statistics']: 
                pd2['View'][i] = int(response['items'][0]['statistics']['viewCount']) 
            else:
                pd2['View'][i] = 0
            #좋아요 수 저장    
            if 'likeCount' in response['items'][0]['statistics']: 
                pd2['Like'][i] = int(response['items'][0]['statistics']['likeCount']) 
            else:
                pd2['Like'][i] = 0
            
            #싫어요 수 저장
            if 'dislikeCount' in response['items'][0]['statistics']: 
                pd2['Dislike'][i] = int(response['items'][0]['statistics']['dislikeCount']) 
            else:
                pd2['Dislike'][i] = 0            
            
            #댓글 수 저장
            if 'commentCount' in response['items'][0]['statistics']: 
                pd2['Comment'][i] = int(response['items'][0]['statistics']['commentCount']) 
            else:
                pd2['Comment'][i] = 0
    return pd2
                        

In [4]:
#종목에 대해 날짜별 주식 종목 관련 동영상 조회수 기준 최대 5개 영상 ID 추출
stockName = ["애플"]

month = [x for x in range(8,9)]
date = [x for x in range(25,32)]

df = pd.DataFrame(columns=['Stock','Date','Video ID','View','Like','Dislike','Comment'])

videosTitles = []
videosID = []
index = 0
dayResult = 10

for i in range(len(stockName)*len(month)*len(date)*dayResult):
    df= df.append({'Stock' : '' , 'Date' : '', 'Video ID': '','View' : 0 , 'Like' : 0, 'Dislike' : 0, 'Comment' : 0} , ignore_index=True)

df = df.astype({'View': np.int64,'Like': np.int64,'Dislike': np.int64,'Comment': np.int64})
for i in stockName:
    for j in month:
        for k in date:
            stockVideoIDExtract(df,index,i,j,k,dayResult)
            YoutubeStatData = stockVideoStatExtract(df)
            index = index + dayResult

In [5]:
YoutubeStatData

,Stock,Date,Video ID,View,Like,Dislike,Comment
0,애플,2020-08-25,J2YTdDwyjEU,33342.0,724.0,0.0,69.0
1,애플,2020-08-25,5keQg1BUuBE,10917.0,294.0,0.0,7.0
2,애플,2020-08-25,Km-p-8bO0l0,7382.0,353.0,0.0,19.0
3,애플,2020-08-25,nO4oE2Pzqx4,5466.0,254.0,0.0,89.0
4,애플,2020-08-25,9Kq0Ven_mqI,798.0,25.0,0.0,10.0
5,애플,2020-08-25,-yRQ-G4fS9g,63.0,1.0,0.0,2.0
6,,,,NaN,NaN,NaN,NaN
7,,,,NaN,NaN,NaN,NaN
8,,,,NaN,NaN,NaN,NaN
9,,,,NaN,NaN,NaN,NaN


In [6]:
for i in range(len(YoutubeStatData)):
    #nan인 경우 제거
    if(YoutubeStatData['View'][i] != YoutubeStatData['View'][i]):
        YoutubeStatData.drop(index=i, axis=0,inplace = True)
    #조회수 1만 이상을 만족하지 않는 동영상 지표 제거     
    else:
        if(int(YoutubeStatData['View'][i])<10000):
            YoutubeStatData.drop(index=i, axis=0,inplace = True)
YoutubeStatData

,Stock,Date,Video ID,View,Like,Dislike,Comment
0,애플,2020-08-25,J2YTdDwyjEU,33342.0,724.0,0.0,69.0
1,애플,2020-08-25,5keQg1BUuBE,10917.0,294.0,0.0,7.0
10,애플,2020-08-26,mbUHXrtYGJU,33245.0,954.0,0.0,111.0
11,애플,2020-08-26,85-Xm-sYlYw,11396.0,288.0,0.0,26.0
20,애플,2020-08-27,VtNxCBBMP3U,37195.0,1108.0,0.0,218.0
21,애플,2020-08-27,EfHCN60h3Gc,24209.0,1708.0,0.0,127.0
22,애플,2020-08-27,aJThilm5wbI,21333.0,1834.0,0.0,131.0
23,애플,2020-08-27,8ZnyCqhCvnU,14881.0,609.0,0.0,56.0
30,애플,2020-08-28,Vpj5tCjUzzs,70932.0,1391.0,0.0,153.0
50,애플,2020-08-30,JSo2GZdaDHs,455563.0,6242.0,0.0,616.0
